<a href="https://colab.research.google.com/github/PriyankaGPawar/MachineLearningWith_Python/blob/master/EDA/COVID_19_Coronavirus_EDA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import plotly.express as px
from collections import namedtuple

In [0]:
#Import data
df=pd.read_csv('/content/covid_19_data.csv', header=0,parse_dates=['Last Update'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22835 entries, 0 to 22834
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   SNo              22835 non-null  int64         
 1   ObservationDate  22835 non-null  object        
 2   Province/State   11006 non-null  object        
 3   Country/Region   22835 non-null  object        
 4   Last Update      22835 non-null  datetime64[ns]
 5   Confirmed        22835 non-null  float64       
 6   Deaths           22835 non-null  float64       
 7   Recovered        22835 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 1.4+ MB


In [6]:
df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0


In [0]:
df.rename(columns={'Province/State':'State','Country/Region':'Country','ObservationDate':'Date'}, inplace=True )

In [8]:
df.head()

,SNo,Date,State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0


In [9]:
df.Deaths.sample(10)

7581        0.0
4191        0.0
9746        0.0
10664       0.0
6722     3130.0
20109       3.0
19472     275.0
17135      14.0
6361        0.0
5837        0.0
Name: Deaths, dtype: float64

In [10]:
df[df['Country']=='India'].sort_values(by=['Confirmed','Date'], ascending=False)

,SNo,Date,State,Country,Last Update,Confirmed,Deaths,Recovered
22587,22588,05/08/2020,NaN,India,2020-05-09 02:32:35,59695.0,1985.0,17887.0
22264,22265,05/07/2020,NaN,India,2020-05-08 02:32:32,56351.0,1889.0,16776.0
21941,21942,05/06/2020,NaN,India,2020-05-07 02:32:28,52987.0,1785.0,15331.0
21618,21619,05/05/2020,NaN,India,2020-05-06 02:32:31,49400.0,1693.0,14142.0
21295,21296,05/04/2020,NaN,India,2020-05-05 02:32:34,46437.0,1566.0,12847.0
...,...,...,...,...,...,...,...,...
672,673,02/03/2020,NaN,India,2020-02-03 21:43:02,3.0,0.0,0.0
607,608,02/02/2020,NaN,India,2020-02-02 06:03:08,2.0,0.0,0.0
547,548,02/01/2020,NaN,India,2020-01-31 08:15:00,1.0,0.0,0.0
491,492,01/31/2020,NaN,India,2020-01-31 23:59:00,1.0,0.0,0.0


In [11]:
df[df.State.notnull()]['Country'].where

<bound method NDFrame.where of 0        Mainland China
1        Mainland China
2        Mainland China
3        Mainland China
4        Mainland China
              ...      
22830                US
22831    Mainland China
22832            Canada
22833    Mainland China
22834    Mainland China
Name: Country, Length: 11006, dtype: object>

In [12]:
df[df['State'].notnull()]['Country'].sample(20)

4020     Mainland China
22135    Mainland China
5593                 UK
1149     Mainland China
15978            France
2518                 US
14707                US
14064                US
9714                 US
6191     Mainland China
20846                US
17960            Canada
8088             Canada
11510            France
19545            Canada
6267                 US
4664                 US
5015             Canada
5374                 US
3077                 US
Name: Country, dtype: object

In [13]:
df.groupby('Country')

In [14]:
df[df['Country']=='US'].sort_values(by='Confirmed', ascending=False)

,SNo,Date,State,Country,Last Update,Confirmed,Deaths,Recovered
22779,22780,05/08/2020,New York,US,2020-05-09 02:32:35,330407.0,26243.0,0.0
22456,22457,05/07/2020,New York,US,2020-05-08 02:32:32,327469.0,26144.0,0.0
22133,22134,05/06/2020,New York,US,2020-05-07 02:32:28,323978.0,25623.0,0.0
21810,21811,05/05/2020,New York,US,2020-05-06 02:32:31,321192.0,25124.0,0.0
21487,21488,05/04/2020,New York,US,2020-05-05 02:32:34,318953.0,24999.0,0.0
...,...,...,...,...,...,...,...,...
4707,4708,03/10/2020,Arkansas,US,2020-03-10 02:33:04,0.0,0.0,0.0
4708,4709,03/10/2020,Delaware,US,2020-03-10 02:33:04,0.0,0.0,0.0
4709,4710,03/10/2020,Idaho,US,2020-03-10 02:33:04,0.0,0.0,0.0
4710,4711,03/10/2020,Maine,US,2020-03-10 02:33:04,0.0,0.0,0.0


In [15]:
df.describe(exclude=None)

,SNo,Confirmed,Deaths,Recovered
count,22835.000000,22835.000000,22835.000000,22835.000000
mean,11418.000000,4513.704839,291.528706,1269.522137
std,6592.041034,20181.863104,1927.577333,8681.169078
min,1.000000,0.000000,0.000000,0.000000
25%,5709.500000,14.000000,0.000000,0.000000
50%,11418.000000,144.000000,2.000000,4.000000
75%,17126.500000,1005.000000,16.000000,124.000000
max,22835.000000,330407.000000,31241.000000,198993.000000


In [16]:
df.isnull().sum() /len(df) *100                 # Province/State column 51% nulls     

SNo             0.000000
Date            0.000000
State          51.802058
Country         0.000000
Last Update     0.000000
Confirmed       0.000000
Deaths          0.000000
Recovered       0.000000
dtype: float64

In [17]:
df2 = df.groupby(["Date", "Country", "State"])[['SNo', 'Date', 'State', 'Country', 'Confirmed', 
                                                'Deaths', 'Recovered']].sum().reset_index()

df2

,Date,Country,State,SNo,Confirmed,Deaths,Recovered
0,01/22/2020,Hong Kong,Hong Kong,13,0.0,0.0,0.0
1,01/22/2020,Macau,Macau,21,1.0,0.0,0.0
2,01/22/2020,Mainland China,Anhui,1,1.0,0.0,0.0
3,01/22/2020,Mainland China,Beijing,2,14.0,0.0,0.0
4,01/22/2020,Mainland China,Chongqing,3,6.0,0.0,0.0
...,...,...,...,...,...,...,...
10997,05/08/2020,US,Virginia,22826,22342.0,812.0,0.0
10998,05/08/2020,US,Washington,22827,16388.0,905.0,0.0
10999,05/08/2020,US,West Virginia,22828,1323.0,52.0,0.0
11000,05/08/2020,US,Wisconsin,22830,9590.0,384.0,0.0


In [18]:
df2[df2['Country']=='US'][['SNo', 'Date','Country','State','Confirmed','Deaths', 'Recovered']]

,SNo,Date,Country,State,Confirmed,Deaths,Recovered
34,32,01/22/2020,US,Washington,1.0,0.0,0.0
69,70,01/23/2020,US,Washington,1.0,0.0,0.0
103,119,01/24/2020,US,Chicago,1.0,0.0,0.0
104,118,01/24/2020,US,Washington,1.0,0.0,0.0
138,160,01/25/2020,US,Illinois,1.0,0.0,0.0
...,...,...,...,...,...,...,...
10997,22826,05/08/2020,US,Virginia,22342.0,812.0,0.0
10998,22827,05/08/2020,US,Washington,16388.0,905.0,0.0
10999,22828,05/08/2020,US,West Virginia,1323.0,52.0,0.0
11000,22830,05/08/2020,US,Wisconsin,9590.0,384.0,0.0


In [37]:
df.query('Country=="Mainland China"').groupby("Last Update")[['Confirmed', 'Deaths', 'Recovered','Country']].sum().sort_values(by='Last Update', ascending=False)

,Confirmed,Deaths,Recovered
Last Update,,,
2020-05-09 02:32:35,82887.0,4633.0,77993.0
2020-05-08 02:32:32,82886.0,4633.0,77993.0
2020-05-07 02:32:28,82885.0,4633.0,77957.0
2020-05-06 02:32:31,82883.0,4633.0,77911.0
2020-05-05 02:32:34,82881.0,4633.0,77853.0
...,...,...,...
2020-01-26 16:00:00,2062.0,56.0,49.0
2020-01-25 17:00:00,1399.0,42.0,39.0
2020-01-24 17:00:00,916.0,26.0,36.0


In [45]:
df.query('Country=="Mainland China"').groupby("Last Update")